<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Deshui Yu      Liangjing Yang
#### Student ID: 34253599      34060871

Date: 21/08/2024


Environment: Python xxxx

Libraries used:
* re (for regular expression, installed and imported) 
* pandas (for data manipulation) 
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 15 `.txt` files and 1 `.xlsx` which included various information about user reviews. In particular, Gmap_id(Google Maps identifier for the review location.),Pictures(Links to related pictures (if available).),UserId(Identifier of the user who posted the review.),Date(Timestamp),rate(Numerical rating),review(the user review),name(review poster)

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** to manage and analyze data.
* **datetime** to handle dates and times.

In [1]:
import re
import pandas
import pandas as pd
from datetime import datetime
# My assignment is running locally, and I have already downloaded the data to my local system, so I don't need to link to Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, open all the files and examine the data within them.

Having examined the file content, the following observations were made:

Examining the XLSX File:
- By opening the XLSX file, I discovered that there are sheets numbered from 0 to 14. This means I need to combine these sheets into a single DataFrame.
- By observing the different sheets, I noticed that some of them contain entire rows and columns of empty values. I think I need to remove these empty values.
- After removing the irrelevant empty rows and columns, I found that all sheets have the same content. However, the format of the date and other data does not meet the requirements of the assignment.

Examining the TXT Files:
- I have 15 files numbered from 0 to 14. The first line of each TXT file contains the label `<?xml version="1.0" encoding="UTF-8"?>`, which is a common point. I can use this to split the data.
- I also noticed that the label `</dataset><record>` can be used to split the data for each user, and this pattern is consistent across all TXT files.
- In the TXT files, each piece of data is wrapped in tags, such as userid, username, rate, pictures, etc. Additionally, the tags are quite varied; for instance, the rate data can have multiple tag forms such as `<Rate>`, `</rate>`, `<rating>`, `</Rating>`, and more. This means that if I want to extract the data, my regular expressions will need to be compatible with these different tag forms.


-------------------------------------

<div class="alert alert-block alert-warning"> 

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. After reading the file information, the data is segmented because it appears concatenated when extracted from the .txt files. Regular expressions are then used to filter the same types of data, which are subsequently stored in a dataframe. Next, the .xlsx file is read, and the data is processed according to the assignment's requirements. Finally, the data is concatenated, and files are exported and generated according to the specifications of the assignment.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently.  It helps in filtering out and retrieving specific types of data from the raw data read from files.

In [2]:
user_id_expression = r"<[^>]*?ser[^>]*?>\s*(\d{21})\s*<[^>]*?/[^>]*?ser[^>]*?>"
time_expression = r"(?:<.{1,3}ime\s*/?>\s*|<.{0,2}[Dd]ate\s*/?>\s*)(\d+)(?:\s*<\s*/\s*.{1,3}ime\s*/?>|<\s*/\s*.{0,2}[Dd]ate\s*/?>)"
rate_expression = r"(?:<.{0,2}[Rr]ate>\s*)(\d+)(?:<.{0,2}[Rr]ate>)"
rateing_expression = r"(?:<.{0,2}[Rr]ating>\s*)(\d+)(?:<.{0,2}[Rr]ating>)"
text_expression = r"(?:<.{0,3}[Tt]ext>\s*)(.*?)(?:<.{0,3}[Tt]ext>)"
review_expression = r"(?:<.{0,2}[Rr]eview>\s*)(.*?)(?:<.{0,2}[Rr]eview>)"
name_expression = r"(?:<.{0,7}[Nn]ame>\s*)(.*?)(?:<.{0,7}[Nn]ame\s*>)"
response_expression = r"(?:<.{0,3}[Rr]esp.{0,4}>\s*)(.*?)(?:<.{0,3}[Rr]esp.{0,4}>)"
picture_expression = r"(?:<.{0,2}[Pp]ic.{1,5}>\s*)(.*?)(?:<.{0,2}[Pp]ic.{1,5}>)"
gmap_id_expression=r"(?:<.{0,2}[Gg]map.{1,3}>\s*)(.*?)(?:<.{0,2}[Gg]map.{1,3}>)"

These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed. 

#### 4.2.1 Reading txt file
My code and data are stored locally, so I set a path to the text files (./student_group181) and initialized an empty string all_text_data to store the data. Using a list called file_names, I listed 15 text files, ranging from 'group181_0.txt' to 'group181_14.txt'. The code then iterates through each file in a loop, opens them, reads their contents, and appends this content to the all_text_data string. This approach merges the text contents of multiple files into a single string, facilitating subsequent data processing or analysis.

In [3]:
#The path stored the text files
text_path = './student_group181'
#Initializes an empty string to store text data
all_text_data = ""
file_names = [
    'group181_0.txt', 'group181_1.txt', 'group181_2.txt', 'group181_3.txt', 
    'group181_4.txt', 'group181_5.txt', 'group181_6.txt', 'group181_7.txt', 
    'group181_8.txt', 'group181_9.txt', 'group181_10.txt', 'group181_11.txt',
    'group181_12.txt', 'group181_13.txt', 'group181_14.txt'
]

#Read the contents of multiple text files and merge them together
for file_name in file_names:
    file_path = f"{text_path}/{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_text_data += content
        
#The path stored the text files
text_path = './student_group181'
#Initializes an empty string to store text data
all_text_data = ""
file_names = [
    'group181_0.txt', 'group181_1.txt', 'group181_2.txt', 'group181_3.txt', 
    'group181_4.txt', 'group181_5.txt', 'group181_6.txt', 'group181_7.txt', 
    'group181_8.txt', 'group181_9.txt', 'group181_10.txt', 'group181_11.txt',
    'group181_12.txt', 'group181_13.txt', 'group181_14.txt'
]

#Read the contents of multiple text files and merge them together
for file_name in file_names:
    file_path = f"{text_path}/{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_text_data += content

#### 4.2.2 Split Data  and Data Extraction
I first utilized specific XML tags as delimiters to split the string containing all text data (all_text_data) into a list (folder_text). This step is effective due to the consistency of XML tags and the structured nature of the data. Any empty strings encountered at the beginning of the split list are removed to ensure data cleanliness and accuracy in subsequent processing.
Next, I applied regular expressions to further segment each section of text, ensuring accurate classification and separation. These segments are stored in the list final_split_text, which is used later for detailed data extraction.
In the extraction phase, I initialized multiple empty lists to store various types of data, including user IDs, Google Maps IDs, timestamps, ratings, review contents, responses, image links, and usernames. This organization facilitates structured storage of the extracted data.
Using precompiled regular expression patterns, I traversed all the text in final_split_text, extracting the required information through precise pattern matching. Each successful match resulted in the corresponding data being appended to its respective list. This approach ensures efficiency and accuracy in data extraction, preparing it for further analysis and application.

In [ ]:
split_txt = '?xml version="1.0" encoding="UTF-8"?>'
final_split= '</dataset><record>|<dataset><record>'

#All text data is divided by XML to get a list  
folder_text = all_text_data.split(split_txt)

#Delete an empty string
if folder_text[0] == "":
    folder_text.pop(0)

#Split the text again according to the regular expression
final_split_text = []
for text in folder_text:
    slipted_info = re.split(final_split,text)
    final_split_text.extend(slipted_info)
    
#Creates an empty list for storing text
user_id_list = []
gmap_id_list = []
time_list = []
date_list = []
rate_list = []
review_list = []
response_list = []
picture_list = []
name_list = []

#Use regular expression compilations to match specific text structures
user_id_pattern = re.compile(user_id_expression, re.DOTALL)
time_pattern = re.compile(time_expression, re.DOTALL)
rate_pattern = re.compile(rate_expression, re.DOTALL)
rateing_pattern = re.compile(rateing_expression, re.DOTALL)
text_pattern = re.compile(text_expression, re.DOTALL)
review_pattern = re.compile(review_expression, re.DOTALL)
response_pattern = re.compile(response_expression, re.DOTALL)
picture_pattern = re.compile(picture_expression, re.DOTALL)
name_pattern = re.compile(name_expression,re.DOTALL)
gmap_id_pattern = re.compile(gmap_id_expression, re.DOTALL)

#Walk through all the segmented text, matching and extracting the corresponding text
for text in final_split_text:
     
     user_id_matches = user_id_pattern.findall(text)
     user_id_list.extend(user_id_matches)
     
     gmap_id_matches = gmap_id_pattern.findall(text)
     gmap_id_list.extend(gmap_id_matches)
     
     time_matches = time_pattern.findall(text)
     time_list.extend(time_matches)
     
     rate_matches = rate_pattern.findall(text)
     if not rate_matches:
        rate_matches = rateing_pattern.findall(text)
     rate_list.extend(rate_matches)
     
     review_matches = review_pattern.findall(text)
     if not review_matches:
        review_matches = text_pattern.findall(text)
     review_list.extend(review_matches)
     
     response_matches = response_pattern.findall(text)
     response_list.extend(response_matches)
     
     picture_matches = picture_pattern.findall(text)
     picture_list.extend(picture_matches)
     
     
     name_matches = name_pattern.findall(text)
     name_list.extend(name_matches)

#### 4.2.3 DataFrame data processing
In this step, a DataFrame named text_datafram was created to store the information from the previous step. Then, according to the requirements for exporting file data for the assignment, the data type of the 'rating' column was converted to a floating point number, and the timestamp data was converted into a readable date-time format. Special strings labeled “None” were replaced with actual None values. The code iterates through each row of the DataFrame, checking the contents of the 'pics' and 'resp' columns to determine the presence of pictures and responses, and the results are added to new columns if_pic and if_response, respectively. Additionally, the function extract_dimensions extracts the width and height from the picture URLs and stores these dimensions in a new column, pic_dim, to meet the assignment's specifications.

In [ ]:
#Create a dataframe from the extracted data
text_datafram = pd.DataFrame({
    'user_id': user_id_list,
    'name':name_list,
    'time': time_list,
    'rating': rate_list,
    'text': review_list,
    'pics': picture_list,
    'resp': response_list,
    'gmap_id': gmap_id_list
})

#Arranges the column order of the dataframe
text_datafram = text_datafram[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]

#Converts the data type of the rating column to float
text_datafram['rating'] = text_datafram['rating'].astype(float)

#Converts the timestamp of the time column to a date-time format and to a string
text_datafram['time'] = pd.to_datetime(pd.to_numeric(text_datafram['time']), unit='ms', utc=True)
text_datafram['time'] = text_datafram['time'].dt.strftime("%Y-%m-%d %H:%M:%S")

#Replace the string 'None' in the data box with the actual None value
text_datafram = text_datafram.replace('None', None)

#Iterate through each row of the data box and determine whether there are pictures and replies based on the contents of the 'pics' and 'resp' columns
if_pic = []
if_response = []
for index, row in text_datafram.iterrows():
    if row['pics'] is None:
        if_pic.append('N')
    else:
        if_pic.append('Y')
    
    if row['resp'] is None:
        if_response.append('N')
    else:
        if_response.append('Y')
text_datafram['if_pic'] = if_pic
text_datafram['if_response'] = if_response

#Defines a function to extract the width and height of the picture from the picture URL and store it in the 'pic_dim' column
def extract_dimensions(text):
    if pd.isna(text):
        return "[ ]"
    dimensions = re.findall(r'w(\d{1,3})-h(\d{1,3})-k-no-p', text)
    if not dimensions:
        return "[ ]"
    return [[int(width), int(height)] for width, height in dimensions]

text_datafram['pic_dim'] = text_datafram['pics'].apply(extract_dimensions)

#### 4.2.5 Read xlsx and data processing
Since the operation is conducted locally, I first specify the path to the Excel file and use the pandas library to read the file. I load the data from each worksheet, removing rows and columns that are entirely empty. Non-empty dataframes are added to the cleaned_sheets list. These cleaned dataframes are then merged into a single dataframe, xls_data. Timestamps are converted to a readable date-time format and formatted as strings according to the assignment requirements. In the 'pics' and 'resp' columns, all missing values are replaced with Python's None. I iterate through each row, determining the presence of pictures and responses based on the contents of the 'pics' and 'resp' columns, and record the results in newly added columns if_pic and if_response. I define a function extract_dimensions to extract the width and height from picture URLs and store them in the new column pic_dim.

In [4]:
#Define the path to the Excel file and read the Excel file
xls_path = "./student_group181/group181.xlsx"
xls = pd.ExcelFile(xls_path)

#Read data from all worksheets in an Excel file
all_sheets = pd.read_excel(xls_path, sheet_name=None)

#Iterate through each worksheet, removing all empty rows and columns and adding them to the list
cleaned_sheets = []
for sheet_name, df in all_sheets.items():
    df = df.dropna(how='all')
    df = df.dropna(axis=1, how='all')
    if not df.empty:
        cleaned_sheets.append(df)

# Merge all cleaned dataframes into one
xls_data = pd.concat(cleaned_sheets, ignore_index=True)
xls_data = xls_data[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]

#Conversion time format
xls_data['time'] = pd.to_datetime(xls_data['time'], unit ='ms', utc = True)

#Format the time column into string format
xls_data['time'] = xls_data['time'].dt.strftime("%Y-%m-%d %H:%M:%S")

#Replace the missing values in the 'pics' and 'resp' columns with None
xls_data['pics'] = xls_data['pics'].where(pd.notna(xls_data['pics']), None)
xls_data['resp'] = xls_data['resp'].where(pd.notna(xls_data['resp']), None)

#Iterate through each row of the data box and determine whether there are pictures and replies based on the contents of the 'pics' and 'resp' columns
if_pic_xls = []
if_response_xls = []
for index, row in xls_data.iterrows():
    if row['pics'] is None:
        if_pic_xls.append('N')
    else:
        if_pic_xls.append('Y')
    
    if row['resp'] is None:
        if_response_xls.append('N')
    else:
        if_response_xls.append('Y')

xls_data['if_pic'] = if_pic_xls
xls_data['if_response'] = if_response_xls

#Defines a function to extract the width and height of the picture from the picture URL and store it in the 'pic_dim' column
def extract_dimensions(text):
    if pd.isna(text):
        return "[ ]"
    dimensions = re.findall(r'w(\d{1,3})-h(\d{1,3})-k-no-p', text)
    if not dimensions:
        return "[ ]"
    return [[int(width), int(height)] for width, height in dimensions]

xls_data['pic_dim'] = xls_data['pics'].apply(extract_dimensions)

To view the data in text_datafram and xls_data and ensure accuracy for subsequent data merging

In [ ]:
text_datafram.info()
xls_data.info()

In [10]:
print(text_datafram.head())
print(xls_data.head())

                 user_id             name                 time  rating  \
0  103499789493991350231   Alyson Hoffman  2021-06-01 04:57:49     5.0   
1  105404622065259965715     OnlyOneEarth  2021-03-21 07:59:56     5.0   
2  118347770680889723845  matthew thomson  2021-04-11 15:44:41     5.0   
3  109541606733835908756          Becky B  2021-03-15 15:42:06     5.0   
4  108792737827746934756  Tiffany Klitzke  2021-02-02 18:00:03     5.0   

                                                text  \
0  Booked last minute rentals here over Memorial ...   
1  When I went into Epicenter Cycling, I was look...   
2  Amazing selection of Trek bikes! We were told ...   
3  Absolutely flawless experience ordering, sizin...   
4  All of the employees were so friendly and help...   

                                                pics  \
0                                               None   
1  [{'url': ['https://lh5.googleusercontent.com/p...   
2                                               No

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Whatever else <a class="anchor" name="latin"></a>

I merged data extracted from Excel and text files into a new dataframe named combined_data, using the pd.concat method to append xls_data and text_datafram vertically (axis=0). Subsequently, I employed a regular expression to remove special characters and emoticons from the text column. This was achieved by applying a function that checks for string instances and then strips out a specified range of Unicode emoticon characters. All text was then converted to lowercase to standardize the data format for easier processing. Finally, the combined dataframe was exported to a CSV file combined_data.csv without saving the row indices. This streamlined process ensures data consistency and prepares it for further analysis.

In [5]:
#Merge data from excel files and txt files
combined_data = pd.concat([xls_data, text_datafram], axis=0)

#Remove special characters and emoticons from text with regular expressions
combined_data['text'] = combined_data['text'].apply(
    lambda x: re.sub(
        r"[\U0001F600-\U0001F64F"  
        r"\U0001F300-\U0001F5FF"  
        r"\U0001F680-\U0001F6FF"  
        r"\U0001F1E0-\U0001F1FF"  
        r"\U00002702-\U000027B0"  
        r"\U000024C2-\U0001F251"  
        r"]+", '', x) if isinstance(x, str) else x
)

#Convert all text to lowercase
combined_data['text'] = combined_data['text'].str.lower()
combined_data.to_csv('combined_data.csv', index=False)

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 5.  Writing to JSON File <a class="anchor" name="write"></a>

</div>

### 5.1 Create csv File
The combined_data dataframe is grouped by the gmap_id column, and aggregated to calculate the total number of reviews, the count of reviews containing text, and the count of reviews with responses for each group. The agg method is used for these calculations, where review_count measures the total reviews per group, review_text_count counts non-empty text fields, and response_count tallies non-empty response fields. The results are reset using reset_index for easier data access in subsequent steps. Finally, the processed data is saved to a CSV file named task1_181.csv, with index=False to prevent the index from being written into the file, ensuring a cleaner file content.

In [ ]:
#Group the data boxes by the 'gmap_id' column and count the amount of corresponding text
grouped = combined_data.groupby('gmap_id').agg(
    review_count=('gmap_id', 'size'),
    review_text_count=('text', lambda x: x.notna().sum()),
    response_count=('resp', lambda x: x.notna().sum())
).reset_index()

#Store data as CSV file
grouped.to_csv('task1_181.csv', index=False)

### 5.2 Create json File
I created a new DataFrame, data_selected, and grouped the data by gmap_id to iterate through it. If picture information is present, the pic_dim column data is converted from list format to string format. For each group, I constructed a list of reviews that includes each review's user ID, time, rating, text, presence of a picture, picture dimensions, and whether there was a response (as required by the assignment). Each gmap_id and its corresponding list of reviews, along with the earliest and latest review dates for that group, are stored in the dictionary output_data. Finally, I saved the output_data dictionary in JSON format to a file named task1_181.json, using UTF-8 encoding and formatted output to enhance readability. This process effectively transforms structured data into a JSON format for further data sharing and processing.

In [7]:
import json
#Select the specified column and create a new DataFrame
data_selected = combined_data[['gmap_id', 'user_id', 'time', 'rating', 'text', 'if_pic', 'pic_dim', 'if_response','pics']]

#Groups the data by 'gmap_id', traverses the rows of data within each group, and converts the data in pic_dim to a string if picture information is present
output_data = {}
for gmap_id, group in data_selected.groupby('gmap_id'):
    reviews = []
    earliest_review_date = group['time'].min()
    latest_review_date = group['time'].max()
    for _, row in group.iterrows():
        pic_dim_converted = [[str(dim) for dim in dims] for dims in row['pic_dim']] if pd.notna(row['pics']) else []

        #Build the structure of a single review
        review = {
            "user_id": row['user_id'],
            "time": row['time'],
            "review_rating": row['rating'],
            "review_text": row['text'],
            "if_pic": row['if_pic'],
            'pic_dim': pic_dim_converted,
            "if_response": row['if_response']
        }
        #Add review data to the reviews list
        reviews.append(review)
    #Store each gmap_id and its corresponding comment list in output_data
        output_data[gmap_id] = {
        "reviews": reviews,
        "earliest_review_date": earliest_review_date,
        "latest_review_date": latest_review_date
    }

# Store data as json file
with open('task1_181.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False, separators=(',', ': '))


....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated JSON File <a class="anchor" name="test_xml"></a>

According to the task1_test.py shared by the tutor, I verified my JSON data, and my data file passed the test.

<div class="alert alert-block alert-warning"> 

## 6. Summary <a class="anchor" name="summary"></a>

</div>

....

In this assessment, I processed data from multiple TXT and XLSX files, merging them into a unified DataFrame. The TXT files were parsed using regular expressions to extract relevant information, while the XLSX sheets were cleaned and combined. The data was then standardized, with special characters and emoticons removed, text converted to lowercase, and timestamps formatted. I grouped and aggregated the data by gmap_id, calculating review counts and response presence, and saved the results into CSV and JSON formats for further analysis. This process ensured clean, consistent data ready for analysis.

....

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 30/08/2022.

We use ChatGPT to help us improve text expression, correct grammar, provide assignment step suggestions, offer code advice, debug code, and check and enhance regular expressions.https://chatgpt.com/


## --------------------------------------------------------------------------------------------------------------------------